# 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

news_data = pd.read_csv("19.01~20.09.csv", encoding="cp949")
news_data.head(5)

In [ ]:
news_data.info()

# 데이터 전처리

## 1)본문이 200자가 안되는 뉴스들 제거
\[인사], \[부고] 등 우리에세 필요없는 내용들은 전부 짧았기때문

## 2)사용자지정 stopword 제거

<코>, (유), \% -> 이런게 title 에 들어가 있으면 주식 관련 내용이다.

In [ ]:
for idx in range(len(news_data)):
    if len(news_data['raw_stream'][idx]) < 200:
        news_data['raw_stream'][idx] = np.nan       
    if news_data['title'][idx].find('%') != -1:
        news_data['title'][idx] = np.nan
    elif news_data['title'][idx].find('코', 1) == 1:
        news_data['title'][idx] = np.nan
    elif news_data['title'][idx].find('코', 1) == 1:
        news_data['title'][idx] = np.nan
    elif news_data['title'][idx].find('유', 1) == 1:
        news_data['title'][idx] = np.nan
    elif news_data['title'][idx].find('유', 1) == 1:
        news_data['title'][idx] = np.nan

## 3) dropna 로 필요없는 행을 다 지운다.

In [ ]:
news_data = news_data.dropna(axis=0)
news_data.info()

## 행을 지웠기 때문에 index 를 재설정해준다.

In [ ]:
news_data = news_data.reset_index()

# Doc2Vec 모델 학습하기

## 1)TaggedDocument 만들기

Doc2Vec 를 돌리기 위해서 TaggedDocument 형태로 만들어 준다.

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(words=[word.lower() for word in _d.split(" ")], 
                              tags=[str(i)]) for i, _d in enumerate(news_data['raw_stream'])]

In [ ]:
len(tagged_data)

## 2) 모델 학습하기
모델을 학습하여 가중치를 d2v.model 파일로 저장해둔다.

In [ ]:
max_epochs = 50
vec_size = 100
alpha = 0.025

d2v_model = Doc2Vec(windows = 5,
                size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=10,
                dm =1)
  
d2v_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    d2v_model.train(tagged_data,
                total_examples=d2v_model.corpus_count,
                epochs=d2v_model.iter)
    # decrease the learning rate
    d2v_model.alpha -= 0.0002
    # fix the learning rate, no decay
    d2v_model.min_alpha = d2v_model.alpha

d2v_model.save("d2v.model")
print("Model Saved")

# 외부 본문 넣어보기

## 모델 불러오기

In [ ]:
d2v_model = Doc2Vec.load('d2v.model')

In [ ]:
s4 = '''
네이버는 자사가 보유한 쇼핑 및 지역 비즈니스 관련 빅데이터를 금융데이터거래소에 등록했다고 18일 밝혔다 디지털 뉴딜의 일환이다 즉 네이버 빅데이터와 은행, 카드사 등 금융권의 데이터를 융합하면 새로운 상품 및 서비스 개발 등에 활용할 수 있다 특정 지역 네이버 사용자들이 많이 검색한 비즈니스 키워드와 성별 및 연령대 데이터를 기반으로 새로운 비즈니스를 발굴하는 형태다

네이버 쇼핑 지역 데이터로 디지털 뉴딜 키운다


네이버는 또 연내 네이버 클라우드 데이터 샌드박스도 공개할 예정이다 네이버가 보유하고 있는 자사 데이터 공공데이터 제휴를 통해 확보한 외부 기업의 데이터를 한 데 모은 플랫폼이다. 클라우드 기반으로 보안성을 높인 한편 사용자들이 보다 직관적으로 활용할 수 있도록 데이터 분석도구와 고성능 인프라 등을 함께 제공한다

네이버는 데이터 샌드박스를 통해 텍스트 이미지 등 인공지능 AI 학습용 데이터 쇼핑 지역 검색 등 사용자 행동 데이터 신사업 개발과 공익 연구를 위한 공공성 데이터 등 여러 분야의 방대한 데이터를 제공한다

또 풍부한 데이터 기반으로 AI 혁신 기술 개발 상권 분석 로보어드바이저 개발 공공정책 및 행정시스템 개선 등 다양한 연구개발이 이뤄질 전망이다

현재 네이버는 클라우드 데이터 샌드박스에서 제공할 데이터에 대한 법률적 검토를 진행하고 있다

다음 달 중 비공개테스트 CBT를 실시해 관련 분야 교수진과 함께 데이터 유용성 등을 점검할 예정이다

네이버 클라우드 데이터 샌드박스 정식 출시 이후에는 협력사 국가기관의 제휴를 확대하여 샌드박스 내 데이터 종류도 다양화될 전망이다

한성숙 네이버 대표는 이번 금융데이터거래소 등록과 연내 출시할 네이버 클라우드 데이터 샌드박스를 통해 공개되는 네이버의 방대한 데이터가 중소형업체 SME 의 성장과 관련 산업계 및 연구에 기여하면서 디지털 뉴딜의 마중물이 될 것이라며 앞으로도 가치 있는 데이터를 공개해 우리 사회 발전과 데이터 생태계 활성화에 앞장설 것이라고 말했다.'''

In [ ]:
s4 = s4.replace("\n", "")

In [ ]:
from konlpy.tag import Okt, Kkma

kkma = Kkma()
test_data = kkma.nouns(s4.lower())
print("입력받는 뉴스 : ", test_data)

v1 = d2v_model.infer_vector(test_data)
print("V1_infer", v1)


In [ ]:
doc_sim_lst = d2v_model.docvecs.most_similar(positive=[v1], topn=30)

i = 0
lst = []

for doc_id, sim in doc_sim_lst:
    lst.append(doc_id)
    print(f"Document {doc_id} - similarity: {sim:.5f}")
    i+=1
    if i == 100:
        break

In [ ]:
for idx, i in enumerate(lst):
    num = int(i)
    print("top",idx+1, news_data['section '][num], news_data['title'][num])

# 사용자에게 추천해주기

## 우리 데이터 안의 index 값으로 유사도 추천

In [ ]:
# 일반

def sec_d2v(Doc_num):

    doc_sim_lst = d2v_model.docvecs.most_similar(positive=[d2v_model.docvecs[Doc_num]], topn=100)

    i = 0
    # 학습 데이터 안의 data
    IN_lst = []

    for doc_id, sim in doc_sim_lst:
        if sim < 0.4:
            break
        IN_lst.append(doc_id)
#         print(f"Document {doc_id} - similarity: {sim:.5f}")
        i+=1
        if i == 100:
            break
            
    
    for idx, i in enumerate(IN_lst):
        num = int(i)
#        print(news_data['dateline'][num][:10], news_data['section '][num], news_data['title'][num])

In [ ]:
# 인덱스 값을 넣으세요
sec_d2v(42523)

# 2차 분류가 일치하는 것만 보여주기
## 날짜 sort

dataframe 출력

In [ ]:
def sec_d2v_df(Doc_num):
    first = 0
    
    d_list = []
    s_list = []
    t_list = []
    l_list = []
    
    doc_sim_lst = d2v_model.docvecs.most_similar(positive=[d2v_model.docvecs[Doc_num]], topn=100)

    i = 0
    # 학습 데이터 안의 data
    IN_lst = []

    for doc_id, sim in doc_sim_lst:
        if sim < 0.4:
            break
        IN_lst.append(doc_id)
#         print(f"Document {doc_id} - similarity: {sim:.5f}")
        i+=1
        if i == 100:
            break
            
    
    for idx, i in enumerate(IN_lst):
        num = int(i)
        if first == 0:
            cat = news_data['section '][num]
        first += 1
        if cat == news_data['section '][num]:

            d_list.append(news_data['dateline'][num][:10])
            s_list.append(news_data['section '][num])
            t_list.append(news_data['title'][num])
            l_list.append(news_data['link'][num])
#             print(news_data['dateline'][num][:10], news_data['section '][num], news_data['title'][num])
        
        notdf = {'dateline' : d_list,
               'section ' : s_list,
               'title' : t_list,
               'link' : l_list}
        df = pd.DataFrame(notdf)
    

    return df



In [ ]:
dfdf = sec_d2v_df(9320)
dfdf1 = dfdf.sort_values(by='dateline', ascending=False)
dfdf1[:20]

# 1차 분류 내에서(컴공) 랜덤으로 출력

In [ ]:
import random

doc_list = []
ran_num = random.randint(0,len(news_data))
for i in range(30):
    while ran_num in doc_list:
        ran_num = random.randint(0,len(news_data))
    doc_list.append(ran_num)
print(doc_list)

In [ ]:
d_list = []
s_list = []
t_list = []
l_list = []
for idx, i in enumerate(doc_list):

    num = int(i)
    d_list.append(news_data['dateline'][num][:10])
    s_list.append(news_data['section '][num])
    t_list.append(news_data['title'][num])
    l_list.append(news_data['link'][num])

    notdf = {'dateline' : d_list,
           'section ' : s_list,
           'title' : t_list,
           'link' : l_list}
    df = pd.DataFrame(notdf)
df